In [1]:
!pip install -q -U langchain langchain_community langchain_cerebras

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.2 which is incompatible.
jupyterlab 4.3.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.3 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is inco

In [2]:
import pandas as pd
import requests

User_demographics = pd.read_csv("/kaggle/input/my-new-datastets/Demographics.csv")
Inspired = pd.read_csv("/kaggle/input/my-new-datastets/Inspired.csv")
Liked = pd.read_csv("/kaggle/input/my-new-datastets/Liked.csv")
Rated= pd.read_csv("/kaggle/input/my-new-datastets/Rated.csv")
viewed = pd.read_csv("/kaggle/input/my-data/viewed.csv")
posts = pd.read_csv("/kaggle/input/another-dataset/Posts.csv")

In [3]:
User_demographics

,id,first_name,last_name,username,email,role,profile_url,bio,website_url,instagram-url,...,has_wallet,last_login,share_count,post_count,following_count,follower_count,is_verified,is_online,latitude,longitude
0,1,Michael,Dadzie,afrobeezy,dadziemikke@outlook.com,A,https://assets.socialverseapp.com/profile/afro...,Digitial Nomad | iOS Developer,https://www.github.com/michaeldadzie,https://www.instagram.com/michaeldadziie,...,True,2024-12-07 12:42:06,0,57,8,20,False,True,NaN,NaN
1,2,John,Doe,doey,test@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,NaN,NaN,NaN,...,False,NaN,0,0,0,0,False,False,NaN,NaN
2,3,John,Doe,doeyy,test1@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,NaN,NaN,NaN,...,False,NaN,0,0,0,0,False,False,NaN,NaN
3,4,John,Doe,doeyyy,code#gmail,U,https://cdn-assets.socialverseapp.com/profile/...,NaN,NaN,NaN,...,False,NaN,0,0,0,0,False,False,NaN,NaN
4,5,Sachin,Kinha,kinha,sachinkinha1913@gmail.com,A,https://assets.socialverseapp.com/profile/kinh...,SDE | Competitive Programming | Deserve before...,https://sachinkinha.com,https://www.instagram.com/kinha_sachin?igsh=dn...,...,True,2024-12-09 04:51:50,0,947,17,47,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1015,yjnywzpbwc,privaterelay.appleid.com,yjnywzpbwc_08c9i6l7f,yjnywzpbwc@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/15.png,NaN,NaN,NaN,...,False,NaN,0,0,0,0,False,False,NaN,NaN
996,1016,xkzvdj57xj,privaterelay.appleid.com,xkzvdj57xj_09f7c9l8i,xkzvdj57xj@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/16.png,NaN,NaN,NaN,...,False,2024-10-08 11:21:08,0,0,0,0,False,False,NaN,NaN
997,1017,Nuage,Laboratoire,nuage_08i0l4c5f,xkqrhcbo7am2slbnr23lfpyzy4-00@cloudtestlabacco...,U,https://assets.socialverseapp.com/profile/16.png,NaN,NaN,NaN,...,False,2024-11-20 12:32:38,0,0,0,0,False,False,NaN,NaN
998,1018,2mhdbqqk6q,privaterelay.appleid.com,2mhdbqqk6q_06l6f3i1c,2mhdbqqk6q@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/1.png,NaN,NaN,NaN,...,False,2024-10-09 11:13:44,0,0,0,1,False,False,NaN,NaN


In [3]:
User_demographics.last_login = User_demographics['last_login'].fillna(User_demographics.last_login.mode()[0])
User_demographics.isna().sum()

id                    0
first_name            0
last_name             0
username              0
email                 0
role                  0
profile_url           0
bio                 981
website_url         992
instagram-url       995
youtube_url         999
tictok_url          998
isVerified            0
referral_code       873
has_wallet            0
last_login            0
share_count           0
post_count            0
following_count       0
follower_count        0
is_verified           0
is_online             0
latitude           1000
longitude          1000
dtype: int64

In [6]:
bools = [col for col in User_demographics.select_dtypes(include = 'bool')]
User_demographics[bools] = User_demographics[bools].astype(float)
User_demographics

,id,first_name,last_name,username,email,role,profile_url,bio,website_url,instagram-url,...,has_wallet,last_login,share_count,post_count,following_count,follower_count,is_verified,is_online,latitude,longitude
0,1,Michael,Dadzie,afrobeezy,dadziemikke@outlook.com,A,https://assets.socialverseapp.com/profile/afro...,Digitial Nomad | iOS Developer,https://www.github.com/michaeldadzie,https://www.instagram.com/michaeldadziie,...,1.0,2024-12-07 12:42:06,0,57,8,20,0.0,1.0,NaN,NaN
1,2,John,Doe,doey,test@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,NaN,NaN,NaN,...,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0,NaN,NaN
2,3,John,Doe,doeyy,test1@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,NaN,NaN,NaN,...,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0,NaN,NaN
3,4,John,Doe,doeyyy,code@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,NaN,NaN,NaN,...,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0,NaN,NaN
4,5,Sachin,Kinha,kinha,sachinkinha1913@gmail.com,A,https://assets.socialverseapp.com/profile/kinh...,SDE | Competitive Programming | Deserve before...,https://sachinkinha.com,https://www.instagram.com/kinha_sachin?igsh=dn...,...,1.0,2024-12-09 04:51:50,0,947,17,47,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1015,yjnywzpbwc,privaterelay.appleid.com,yjnywzpbwc_08c9i6l7f,yjnywzpbwc@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/15.png,NaN,NaN,NaN,...,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0,NaN,NaN
996,1016,xkzvdj57xj,privaterelay.appleid.com,xkzvdj57xj_09f7c9l8i,xkzvdj57xj@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/16.png,NaN,NaN,NaN,...,0.0,2024-10-08 11:21:08,0,0,0,0,0.0,0.0,NaN,NaN
997,1017,Nuage,Laboratoire,nuage_08i0l4c5f,xkqrhcbo7am2slbnr23lfpyzy4-00@cloudtestlabacco...,U,https://assets.socialverseapp.com/profile/16.png,NaN,NaN,NaN,...,0.0,2024-11-20 12:32:38,0,0,0,0,0.0,0.0,NaN,NaN
998,1018,2mhdbqqk6q,privaterelay.appleid.com,2mhdbqqk6q_06l6f3i1c,2mhdbqqk6q@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/1.png,NaN,NaN,NaN,...,0.0,2024-10-09 11:13:44,0,0,0,1,0.0,0.0,NaN,NaN


In [7]:
print(User_demographics.columns)
print(Inspired.columns)
print(Liked.columns)
print(Rated.columns)
print(viewed.columns)
print(posts.columns)

print(User_demographics.info())
print(Inspired.info())
print(Liked.info())
print(Rated.info())
print(viewed.info())
print(posts.info())

Index(['id', 'first_name', 'last_name', 'username', 'email', 'role',
       'profile_url', 'bio', 'website_url', 'instagram-url', 'youtube_url',
       'tictok_url', 'isVerified', 'referral_code', 'has_wallet', 'last_login',
       'share_count', 'post_count', 'following_count', 'follower_count',
       'is_verified', 'is_online', 'latitude', 'longitude'],
      dtype='object')
Index(['id', 'post_id', 'user_id', 'inspired_at'], dtype='object')
Index(['id', 'post_id', 'user_id', 'liked_at'], dtype='object')
Index(['id', 'post_id', 'user_id', 'rating_percent', 'rated_at'], dtype='object')
Index(['id', 'post_id', 'user_id', 'viewed_at'], dtype='object')
Index(['id', 'category', 'slug', 'title', 'identifier', 'comment_count',
       'upvote_count', 'view_count', 'exit_count', 'rating_count',
       'average_rating', 'share_count', 'video_link', 'contract_address',
       'chain_id', 'chart_url', 'baseToken', 'is_locked', 'created_at',
       'first_name', 'last_name', 'username', 'upvoted'

In [8]:
posts.isna().sum()

id                     0
category               0
slug                   0
title                322
identifier             0
comment_count          0
upvote_count           0
view_count             0
exit_count             0
rating_count           0
average_rating         0
share_count            0
video_link             0
contract_address     932
chain_id             932
chart_url            932
baseToken              0
is_locked              0
created_at             0
first_name             0
last_name              0
username               0
upvoted                0
bookmarked             0
thumbnail_url          0
gif_thumbnail_url    883
following              0
picture_url            0
post_summary           0
dtype: int64

In [9]:
posts['title'] = posts['title'].fillna(posts['title'].mode()[0])

In [4]:
import pandas as pd
import numpy as np
import re


def load_dataset(file_path):
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        print(f"Error loading dataset: {str(e)}")
        return None


def identify_missing_columns(df):
    missing_columns = df.columns[df.isnull().sum() > 0.5*len(df)].tolist()
    return missing_columns

def fill_missing_values(df, column, value):
    if column in df.columns:
        df[column] = df[column].fillna(value)
    return df

def drop_missing_columns(df, columns):
    if isinstance(columns, list):
        df = df.drop(columns=columns, axis=1)
    return df

def validate_data_formats(df):
    def validate_email(email):
        pattern = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
        return bool(re.match(pattern, email))

    def validate_url(url):
        pattern = r"^https?:\/\/(?:[-\w]+\.)+[a-zA-Z]{2,}(?:\/[^\s]*)?$"
        return bool(re.match(pattern, url))

    def validate_coordinates(lat, lon):
        try:
            float(lat)
            float(lon)
            return float(lat) >= -90 and float(lat) <= 90 and float(lon) >= -180 and float(lon) <= 180
        except ValueError:
            return False

    for index, row in df.iterrows():
        if 'email' in row and not validate_email(row['email']):
            print(f"Email format is invalid for record {index}: {row['email']}")
        if 'profile_url' in row and not validate_url(row['profile_url']):
            print(f"Profile URL format is invalid for record {index}: {row['profile_url']}")
        if 'website_url' in row and not validate_url(row['website_url']):
            print(f"Website URL format is invalid for record {index}: {row['website_url']}")
        if 'instagram-url' in row and not validate_url(row['instagram-url']):
            print(f"Instagram URL format is invalid for record {index}: {row['instagram-url']}")
        if 'youtube_url' in row and not validate_url(row['youtube_url']):
            print(f"YouTube URL format is invalid for record {index}: {row['youtube_url']}")
        if 'tictok_url' in row and not validate_url(row['tictok_url']):
            print(f"Tictok URL format is invalid for record {index}: {row['tictok_url']}")
        if 'latitude' in row and 'longitude' in row and not validate_coordinates(row['latitude'], row['longitude']):
            print(f"Latitude or longitude format is invalid for record {index}: {row['latitude']}, {row['longitude']}")

missing_columns = identify_missing_columns(User_demographics)

print(f"Columns with more than 50% missing values: {missing_columns}")


action = input("Do you want to fill the missing values or drop the columns? (fill/drop): ")

if action.lower() == 'fill':
    for column in missing_columns:
        User_demographics = fill_missing_values(User_demographics, column, value)
    print("Missing values filled successfully.")
elif action.lower() == 'drop':
    confirm = input(f"Are you sure you want to drop columns {missing_columns}? (yes/no): ")
    if confirm.lower() == 'yes':
        User_demographics = drop_missing_columns(User_demographics, missing_columns)
        print("Columns dropped successfully.")
    else:
        print("Operation cancelled.")
else:
    print("Invalid action selected.")

validate_data_formats(User_demographics)

Columns with more than 50% missing values: ['bio', 'website_url', 'instagram-url', 'youtube_url', 'tictok_url', 'referral_code', 'latitude', 'longitude']


Do you want to fill the missing values or drop the columns? (fill/drop):  drop
Are you sure you want to drop columns ['bio', 'website_url', 'instagram-url', 'youtube_url', 'tictok_url', 'referral_code', 'latitude', 'longitude']? (yes/no):  yes


Columns dropped successfully.
Email format is invalid for record 3: code#gmail
Email format is invalid for record 6: codewitha#gmail


In [5]:
User_demographics.loc[3,'email'] = 'code@gmail.com'
User_demographics.loc[6,'email'] = 'codewith@gmail.com'

# Inspecting the unique records present in the datasets

In [6]:
User_demographics.nunique()

id                 1000
first_name          874
last_name           468
username           1000
email               999
role                  2
profile_url          60
isVerified            1
has_wallet            2
last_login          747
share_count           1
post_count            9
following_count      14
follower_count       13
is_verified           1
is_online             2
dtype: int64

In [7]:
User_demographics.last_login = User_demographics['last_login'].fillna(User_demographics.last_login.mode()[0])
User_demographics.isna().sum()

id                 0
first_name         0
last_name          0
username           0
email              0
role               0
profile_url        0
isVerified         0
has_wallet         0
last_login         0
share_count        0
post_count         0
following_count    0
follower_count     0
is_verified        0
is_online          0
dtype: int64

In [8]:
bools = [col for col in User_demographics.select_dtypes(include = 'bool')]
User_demographics[bools] = User_demographics[bools].astype(float)

In [9]:
from kaggle_secrets import UserSecretsClient
from langchain_cerebras import ChatCerebras
from langchain.prompts import ChatPromptTemplate
import pandas as pd
from IPython.display import Markdown,display

user_secrets = UserSecretsClient()
cerebras_api_key = user_secrets.get_secret("CEREBRAS_API_KEY")
llm = ChatCerebras(model="llama3.1-8b", api_key=cerebras_api_key)
prompt = ChatPromptTemplate.from_messages(
    [
("system","""You are given a dataset with the following columns:

- `category`: A string column containing dictionary-like data in some entries, but may also include malformed or invalid values.
- `category_name`: This column will be populated with the 'name' value from the `category` dictionary after it has been safely parsed.
- Other columns with standard data (not relevant for this operation).The name of the datastet is posts which has already been loaded, all you need to do is :

 Go through the dataset and inspect : {data_head} , info : {data_info} and columns of the data : {data_columns}
Your task is to:
1. Clean and preprocess the `category` column by:
   - Removing any escaped single quotes (`\'`) and newline characters (`\n`).
   - Safely parsing valid dictionary strings into actual dictionaries using `ast.literal_eval` or an equivalent method.
   - Handling malformed strings and `NaN` values by returning `None` for invalid entries.
2. Extract the `name` field from the parsed dictionaries and populate the `category_name` column.
3. Return the final resolved code that achieves this transformation, without any comments.

Make sure the final code runs successfully and processes all entries in the `category` column.
""")
    ]
)

posts = pd.read_csv("/kaggle/input/another-dataset/Posts.csv")
formatted_prompt = prompt.format(
    data_columns =posts.columns.tolist(),
    data_head = posts.head(1000),
    data_info = posts.info()
)

response = llm.invoke(formatted_prompt)
display(Markdown(response.content))

def handle_feedback():
    print("\nIf you face any issues with the generated code, you can enter the error or issue below. Press 'Enter' to skip.")
    
    while True:
        user_input = input("Enter your issue or press Enter to continue: ")
        
        # If user presses 'Enter' without input, exit feedback loop
        if not user_input.strip():
            print("No input detected. Skipping feedback.")
            break
        
        # Send the feedback to the LLM for further adjustments
        feedback_prompt = f"User reported an issue: {user_input}. How would you suggest modifying the preprocessing code?"
        
        # Call the LLM again with the user input
        response = llm.invoke(feedback_prompt)
        display(Markdown(response.content))
        
        # Optionally, give the user a chance to retry or end the feedback loop
        retry = input("Would you like to provide more feedback? (yes/no): ")
        if retry.lower() != 'yes':
            break

#
handle_feedback()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 1000 non-null   int64 
 1   category           1000 non-null   object
 2   slug               1000 non-null   object
 3   title              678 non-null    object
 4   identifier         1000 non-null   object
 5   comment_count      1000 non-null   int64 
 6   upvote_count       1000 non-null   int64 
 7   view_count         1000 non-null   int64 
 8   exit_count         1000 non-null   int64 
 9   rating_count       1000 non-null   int64 
 10  average_rating     1000 non-null   int64 
 11  share_count        1000 non-null   int64 
 12  video_link         1000 non-null   object
 13  contract_address   68 non-null     object
 14  chain_id           68 non-null     object
 15  chart_url          68 non-null     object
 16  baseToken          1000 non-null   object
 

```python
import pandas as pd
import ast

def clean_category(row):
    try:
        # Remove escaped single quotes and newline characters
        category_str = row['category'].replace("'", "").replace("\n", "")
        
        # Safely parse valid dictionary strings into actual dictionaries
        category_dict = ast.literal_eval(category_str)
        
        # Return the 'name' field from the parsed dictionary
        return category_dict.get('name')
    except (ValueError, KeyError):
        # Handle malformed strings and NaN values by returning None
        return None

# Apply the cleaning function to the 'category' column
posts['category_name'] = posts.apply(clean_category, axis=1)
```

This code defines a function `clean_category` that takes a row from the DataFrame as input. It attempts to clean and parse the `category` string, and then extracts the `name` field from the resulting dictionary. If the parsing fails (due to a malformed string or NaN value), it returns `None`.

The `clean_category` function is then applied to each row in the DataFrame using the `apply` method, and the resulting values are stored in the new `category_name` column.


If you face any issues with the generated code, you can enter the error or issue below. Press 'Enter' to skip.


Enter your issue or press Enter to continue:  


No input detected. Skipping feedback.


In [10]:
def generate_slug(title):
    return '-'.join(title.lower().strip().split())

posts['slug'] = posts.apply(lambda row: generate_slug(row['title']) if row['slug'] != generate_slug(row['title']) else row['slug'],axis=1)

AttributeError: 'float' object has no attribute 'lower'

In [11]:
import ast
import pandas as pd

def safe_parse_category(x):
    try:
        x = x.replace('\\\'', '\'').replace('\\n', '')
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return None

posts['category'] = posts['category'].apply(safe_parse_category)

posts['category_name'] = posts['category'].apply(lambda x: x['name'] if x else None)


posts = posts.dropna(subset=['category_name'])

print(posts.head())

   id                                           category  \
0  11  {'id': 2, 'name': 'Vible', 'count': 535, 'desc...   
1  12  {'id': 2, 'name': 'Vible', 'count': 535, 'desc...   
2  13  {'id': 2, 'name': 'Vible', 'count': 535, 'desc...   
3  14  {'id': 2, 'name': 'Vible', 'count': 535, 'desc...   
4  15  {'id': 2, 'name': 'Vible', 'count': 535, 'desc...   

                        slug                      title identifier  \
0    recipe-for-a-flow-state    Recipe for a flow state    OSsJAMz   
1                 why-fit-in              Why fit in..?    iGqBOIW   
2   transcending-singularity   Transcending Singularity    QAUE7s4   
3           peak-performance          Peak Performance?    7bKiXIe   
4  our-existential-situation  Our Existential Situation    BLFD9FX   

   comment_count  upvote_count  view_count  exit_count  rating_count  ...  \
0              0            45          63           0             7  ...   
1              0            42          54           0          

In [32]:
posts = posts.drop(columns = ['category'],axis = 1)

In [33]:
posts = posts.drop(columns = ['contract_address','chain_id','chart_url','gif_thumbnail_url'],axis = 1)
posts.isna().sum()

KeyError: "['contract_address', 'chain_id', 'chart_url', 'gif_thumbnail_url'] not found in axis"

In [13]:
posts['title'] = posts['title'].fillna(posts['title'].mode()[0])
posts.isna().sum()

id                0
category          0
slug              0
title             0
identifier        0
comment_count     0
upvote_count      0
view_count        0
exit_count        0
rating_count      0
average_rating    0
share_count       0
video_link        0
baseToken         0
is_locked         0
created_at        0
first_name        0
last_name         0
username          0
upvoted           0
bookmarked        0
thumbnail_url     0
following         0
picture_url       0
post_summary      0
category_name     0
dtype: int64

In [14]:
posts['post_summary'].values[:5]

array(['{\'actions\': {\'key_events\': [\'discussion about flow state\', \'demonstration of body language and gestures\', \'showcasing examples of outdoor activities\']}, \'audio_elements\': {\'descriptions\': [\'synchronized audio and visual elements\', \'animated speaking style\', \'emotional peaks corresponding to content\']}, \'description\': \'The video unfolds as a dynamic exploration of the concept of "flow state," complementing the audio transcription with visual storytelling. The main speaker, a male individual, captures the viewer’s attention through expressive facial gestures and animated speech, embodying the theme of immersion and presence. His body language is energetic and open, illustrating his passion for the subject matter.\\n\\nAs the speaker shares insights on achieving a flow state, the visuals alternate between close-up shots of his face and mid-range views that allow for hand gestures, such as pointing and emphasizing key points. His expressions change from conte

In [15]:
import pandas as pd
import ast

posts['post_summary'] = posts['post_summary'].apply(ast.literal_eval)

posts['description'] = posts['post_summary'].apply(lambda x: x.get('description', 'No description available'))
posts['actions'] = posts['post_summary'].apply(lambda x: x.get('actions', 'No actions available'))
posts['emotions'] = posts['post_summary'].apply(lambda x: x.get('emotions', 'No emotions available'))


posts[['description', 'actions', 'emotions']]

,description,actions,emotions
0,The video unfolds as a dynamic exploration of ...,"{'key_events': ['discussion about flow state',...","{'conveyed': ['enthusiasm', 'engagement', 'cur..."
1,The video is structured as an emotional and in...,{'key_actions': ['Character expressing inner t...,"{'expressed_emotions': ['Anger', 'Determinatio..."
2,The video presents a compelling visual narrati...,{'key_actions': ['Speaker engaging in animated...,"{'emotional_content': ['Wonder', 'Intensity', ..."
3,The video presents a seamless narrative explor...,"{'key_actions': ['speaker uses hand gestures',...","{'moods': ['excitement', 'passion', 'contempla..."
4,"The video opens with a male speaker, who has a...",{'key_actions': ['Speaker engages with the cam...,"{'conveyed_emotions': ['Concern', 'Passion', '..."
...,...,...,...
995,I'm unable to analyze or interpret video frame...,{'main_action': 'suggesting the creation of st...,"{'conveyed_emotions': ['encouraging', 'support..."
996,"As the video unfolds, it presents a cohesive n...",{'key_actions': ['walking through a vast open ...,"{'range_of_emotions': ['deep contemplation', '..."
997,"In the video, a cohesive narrative unfolds thr...","{'key_actions': ['Adjusting headphones', 'Sket...",{'emotional_expressions': ['Focused determinat...
998,The video unfolds as a cinematic narrative set...,{'initial_actions': ['Carrying an old-fashione...,"{'expressed': ['anticipation', 'concern', 'exc..."


In [16]:
posts = posts.drop(columns = 'post_summary',axis = 1)

In [129]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1000 non-null   int64 
 1   slug            1000 non-null   object
 2   title           1000 non-null   object
 3   identifier      1000 non-null   object
 4   comment_count   1000 non-null   int64 
 5   upvote_count    1000 non-null   int64 
 6   view_count      1000 non-null   int64 
 7   exit_count      1000 non-null   int64 
 8   rating_count    1000 non-null   int64 
 9   average_rating  1000 non-null   int64 
 10  share_count     1000 non-null   int64 
 11  video_link      1000 non-null   object
 12  baseToken       1000 non-null   object
 13  is_locked       1000 non-null   bool  
 14  created_at      1000 non-null   int64 
 15  first_name      1000 non-null   object
 16  last_name       1000 non-null   object
 17  username        1000 non-null   object
 18  upvoted  

In [26]:
!pip install -q transformers

In [27]:
viewed['post_id'].unique

<bound method Series.unique of 0      631
1      626
2      571
3      558
4      551
      ... 
995    161
996    143
997    151
998    159
999    560
Name: post_id, Length: 1000, dtype: int64>

In [17]:
Liked['liked_at'] = pd.to_datetime(Liked['liked_at'])
Liked

,id,post_id,user_id,liked_at
0,2,26,9,2023-10-24 19:28:41
1,3,33,9,2023-10-26 09:08:32
2,5,36,1,2023-10-28 09:08:35
3,7,36,16,2023-10-28 09:15:57
4,9,52,9,2023-10-30 15:48:46
...,...,...,...,...
995,1623,1062,912,2024-08-09 13:15:14
996,1625,1104,917,2024-08-09 13:27:55
997,1626,1098,917,2024-08-09 13:28:08
998,1627,1099,1,2024-08-09 16:44:25


In [29]:
Rated['rated_at'] = pd.to_datetime(Rated['rated_at'])
Rated

,id,post_id,user_id,rating_percent,rated_at
0,1,516,1,64,2024-01-11 12:45:41
1,2,160,1,62,2024-01-11 12:47:15
2,3,19,1,20,2024-01-11 12:47:28
3,4,148,1,100,2024-01-11 12:49:35
4,5,513,1,63,2024-01-11 12:50:58
...,...,...,...,...,...
995,1036,142,487,20,2024-06-20 09:58:48
996,1037,516,487,19,2024-06-20 09:58:49
997,1038,150,487,19,2024-06-20 09:58:51
998,1039,779,487,17,2024-06-20 09:58:52


In [18]:
User_demographics

,id,first_name,last_name,username,email,role,profile_url,isVerified,has_wallet,last_login,share_count,post_count,following_count,follower_count,is_verified,is_online
0,1,Michael,Dadzie,afrobeezy,dadziemikke@outlook.com,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,0,57,8,20,0.0,1.0
1,2,John,Doe,doey,test@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
2,3,John,Doe,doeyy,test1@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
3,4,John,Doe,doeyyy,code@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
4,5,Sachin,Kinha,kinha,sachinkinha1913@gmail.com,A,https://assets.socialverseapp.com/profile/kinh...,0.0,1.0,2024-12-09 04:51:50,0,947,17,47,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1015,yjnywzpbwc,privaterelay.appleid.com,yjnywzpbwc_08c9i6l7f,yjnywzpbwc@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/15.png,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
996,1016,xkzvdj57xj,privaterelay.appleid.com,xkzvdj57xj_09f7c9l8i,xkzvdj57xj@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/16.png,0.0,0.0,2024-10-08 11:21:08,0,0,0,0,0.0,0.0
997,1017,Nuage,Laboratoire,nuage_08i0l4c5f,xkqrhcbo7am2slbnr23lfpyzy4-00@cloudtestlabacco...,U,https://assets.socialverseapp.com/profile/16.png,0.0,0.0,2024-11-20 12:32:38,0,0,0,0,0.0,0.0
998,1018,2mhdbqqk6q,privaterelay.appleid.com,2mhdbqqk6q_06l6f3i1c,2mhdbqqk6q@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/1.png,0.0,0.0,2024-10-09 11:13:44,0,0,0,1,0.0,0.0


In [34]:
bools = [col for col in posts.select_dtypes(include = 'bool')]
posts[bools] = posts[bools].astype(float)

In [20]:
print(posts.info())
print(User_demographics.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1000 non-null   int64  
 1   category        1000 non-null   object 
 2   slug            1000 non-null   object 
 3   title           1000 non-null   object 
 4   identifier      1000 non-null   object 
 5   comment_count   1000 non-null   int64  
 6   upvote_count    1000 non-null   int64  
 7   view_count      1000 non-null   int64  
 8   exit_count      1000 non-null   int64  
 9   rating_count    1000 non-null   int64  
 10  average_rating  1000 non-null   int64  
 11  share_count     1000 non-null   int64  
 12  video_link      1000 non-null   object 
 13  baseToken       1000 non-null   object 
 14  is_locked       1000 non-null   float64
 15  created_at      1000 non-null   int64  
 16  first_name      1000 non-null   object 
 17  last_name       1000 non-null   ob

# Merging user demographics and posts with username as common column

In [35]:
combined_df = pd.merge(posts, User_demographics, on='username', how='inner')
combined_df.isna().sum()

id_x               0
slug               0
title              0
identifier         0
comment_count      0
upvote_count       0
view_count         0
exit_count         0
rating_count       0
average_rating     0
share_count_x      0
video_link         0
baseToken          0
is_locked          0
created_at         0
first_name_x       0
last_name_x        0
username           0
upvoted            0
bookmarked         0
thumbnail_url      0
following          0
picture_url        0
category_name      0
description        0
actions            1
emotions           1
id_y               0
first_name_y       0
last_name_y        0
email              0
role               0
profile_url        0
isVerified         0
has_wallet         0
last_login         0
share_count_y      0
post_count         0
following_count    0
follower_count     0
is_verified        0
is_online          0
dtype: int64

In [36]:
combined_df['username'].unique()

array(['afrobeezy', 'mike', 'jack', 'kinha', 'swapnil', 'tarak', 'mridul',
       'ybctjr2q9r_01l6i9f7c', 'daksh_bhardwaj',
       'mohamedmostafffa_07i8f5l1c', 'michaeldadziie', 'shivam'],
      dtype=object)

In [38]:
combined_df.drop(columns=[col for col in combined_df.columns if col.endswith('_y')], inplace=True)
combined_df.rename(columns={'share_count_x': 'share_count', 'first_name_x': 'first_name', 'last_name_x': 'last_name'}, inplace=True)
combined_df.fillna({'role': 'Unknown'}, inplace=True)
combined_df

,id_x,slug,title,identifier,comment_count,upvote_count,view_count,exit_count,rating_count,average_rating,...,role,profile_url,isVerified,has_wallet,last_login,post_count,following_count,follower_count,is_verified,is_online
0,11,recipe-for-a-flow-state,Recipe for a flow state,OSsJAMz,0,45,63,0,7,36,...,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,57,8,20,0.0,1.0
1,12,why-fit-in,Why fit in..?,iGqBOIW,0,42,54,0,0,0,...,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,57,8,20,0.0,1.0
2,13,transcending-singularity,Transcending Singularity,QAUE7s4,0,20,75,0,45,8,...,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,57,8,20,0.0,1.0
3,14,peak-performance,Peak Performance?,7bKiXIe,0,71,95,6,6,28,...,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,57,8,20,0.0,1.0
4,15,our-existential-situation,Our Existential Situation,BLFD9FX,0,64,72,0,9,33,...,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,57,8,20,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1332,ece97b1efa78b1042eaa21170d38f83a006103b6,"Kalsubai is the highest peak in Maharashtra, s...",5A0oaCu,0,23,54,49,2,31,...,U,https://assets.socialverseapp.com/profile/shiv...,0.0,1.0,2024-12-06 06:32:01,19,8,8,0.0,0.0
996,1334,ce76de5e51d2c4590c9f1fd947c1f2dffcb83938,"Tap into the magic of winter. .Sometimes, all ...",15NGhqi,0,7,88,53,2,23,...,U,https://assets.socialverseapp.com/profile/shiv...,0.0,1.0,2024-12-06 06:32:01,19,8,8,0.0,0.0
997,1335,bdc13af36b85c9170d6b7bd29e48009719220441,anzeige -- ️From idea to masterpiece ️with the,ICwNbra,0,24,90,88,2,16,...,U,https://assets.socialverseapp.com/profile/shiv...,0.0,1.0,2024-12-06 06:32:01,19,8,8,0.0,0.0
998,1336,5bcf09ef8e1557548f2b3ca645f034f571d3e420,"In the heart of darkness, magic is our only sh...",IQ-Mqig,0,5,83,42,2,12,...,U,https://assets.socialverseapp.com/profile/shiv...,0.0,1.0,2024-12-06 06:32:01,19,8,8,0.0,0.0


In [39]:
combined_df = combined_df.dropna()
combined_df.isna().sum()

id_x               0
slug               0
title              0
identifier         0
comment_count      0
upvote_count       0
view_count         0
exit_count         0
rating_count       0
average_rating     0
share_count        0
video_link         0
baseToken          0
is_locked          0
created_at         0
first_name         0
last_name          0
username           0
upvoted            0
bookmarked         0
thumbnail_url      0
following          0
picture_url        0
category_name      0
description        0
actions            0
emotions           0
email              0
role               0
profile_url        0
isVerified         0
has_wallet         0
last_login         0
post_count         0
following_count    0
follower_count     0
is_verified        0
is_online          0
dtype: int64

In [40]:
columns_to_drop = [
    'id_x', 'slug', 'identifier', 'video_link', 'baseToken', 'is_locked', 'created_at', 
    'first_name', 'last_name', 'is_verified', 'is_online', 'profile_url', 'last_login','picture_url', 'actions', 'email', 
    'role'
]

filtered_df = combined_df.drop(columns=columns_to_drop)

print(filtered_df.head())

                       title  comment_count  upvote_count  view_count  \
0    Recipe for a flow state              0            45          63   
1              Why fit in..?              0            42          54   
2   Transcending Singularity              0            20          75   
3          Peak Performance?              0            71          95   
4  Our Existential Situation              0            64          72   

   exit_count  rating_count  average_rating  share_count   username  upvoted  \
0           0             7              36            8  afrobeezy      1.0   
1           0             0               0            0  afrobeezy      0.0   
2           0            45               8            0  afrobeezy      0.0   
3           6             6              28            0  afrobeezy      0.0   
4           0             9              33            0  afrobeezy      0.0   

   ...                                      thumbnail_url following  \
0  ...  h

In [41]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 0 to 999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            999 non-null    object 
 1   comment_count    999 non-null    int64  
 2   upvote_count     999 non-null    int64  
 3   view_count       999 non-null    int64  
 4   exit_count       999 non-null    int64  
 5   rating_count     999 non-null    int64  
 6   average_rating   999 non-null    int64  
 7   share_count      999 non-null    int64  
 8   username         999 non-null    object 
 9   upvoted          999 non-null    float64
 10  bookmarked       999 non-null    float64
 11  thumbnail_url    999 non-null    object 
 12  following        999 non-null    float64
 13  category_name    999 non-null    object 
 14  description      999 non-null    object 
 15  emotions         999 non-null    object 
 16  isVerified       999 non-null    float64
 17  has_wallet       999 

In [42]:
import pandas as pd
import ast

# Function to process the emotions column
def extract_emotions(emotion_entry):
    emotions = []
    try:
        if isinstance(emotion_entry, dict):  # If it's a dictionary
            for key, value in emotion_entry.items():
                if isinstance(value, list):
                    emotions.extend(value)
        elif isinstance(emotion_entry, list):  # If it's a list
            for item in emotion_entry:
                if isinstance(item, dict):  # Handle nested dictionaries
                    for key, value in item.items():
                        if isinstance(value, list):
                            emotions.extend(value)
                elif isinstance(item, str):  # Handle string elements in list
                    emotions.append(item)
        elif isinstance(emotion_entry, str):  # If it's a string, parse as dictionary if possible
            parsed_entry = ast.literal_eval(emotion_entry)
            if isinstance(parsed_entry, dict):
                for key, value in parsed_entry.items():
                    if isinstance(value, list):
                        emotions.extend(value)
    except Exception as e:
        print(f"Error processing entry {emotion_entry}: {e}")
    return emotions

# Apply the function to the DataFrame
filtered_df['extracted_emotions'] = filtered_df['emotions'].apply(extract_emotions)

# Display the result
print(filtered_df[['username', 'extracted_emotions']])

      username                                 extracted_emotions
0    afrobeezy       [enthusiasm, engagement, curiosity, passion]
1    afrobeezy  [Anger, Determination, Pain, Empowerment, Isol...
2    afrobeezy  [Wonder, Intensity, Compassion, Engagement, Co...
3    afrobeezy               [excitement, passion, contemplation]
4    afrobeezy  [Concern, Passion, Reflectiveness, Urgency, Ho...
..         ...                                                ...
995     shivam                 [encouraging, supportive, helpful]
996     shivam  [deep contemplation, curiosity, nostalgia, awe...
997     shivam  [Focused determination, Inspiring, Calmness, P...
998     shivam  [anticipation, concern, excitement, fear, exhi...
999     shivam                 [suspense, engagement, excitement]

[999 rows x 2 columns]


In [59]:
filtered_df['thumbnail_url']

0      https://video-cdn.socialverseapp.com/michael_9...
1      https://video-cdn.socialverseapp.com/michael_5...
2      https://video-cdn.socialverseapp.com/michael_5...
3      https://video-cdn.socialverseapp.com/michael_4...
4      https://video-cdn.socialverseapp.com/michael_b...
                             ...                        
995    https://video-cdn.socialverseapp.com/shivam_03...
996    https://video-cdn.socialverseapp.com/shivam_03...
997    https://video-cdn.socialverseapp.com/shivam_03...
998    https://video-cdn.socialverseapp.com/shivam_03...
999    https://video-cdn.socialverseapp.com/shivam_03...
Name: thumbnail_url, Length: 999, dtype: object

In [49]:
filtered_df['emotions_conveyed'] = filtered_df['extracted_emotions']
filtered_df = filtered_df.drop(columns = 'extracted_emotions')

In [48]:
filtered_df['emotions_conveyed'] = filtered_df['emotions_conveyed'].astype(str)
filtered_df['emotions_conveyed'].dtype

dtype('O')

,title,comment_count,upvote_count,view_count,exit_count,rating_count,average_rating,share_count,username,upvoted,...,thumbnail_url,following,category_name,description,isVerified,has_wallet,post_count,following_count,follower_count,emotions_conveyed
0,Recipe for a flow state,0,45,63,0,7,36,8,afrobeezy,1.0,...,https://video-cdn.socialverseapp.com/michael_9...,1.0,Vible,The video unfolds as a dynamic exploration of ...,0.0,1.0,57,8,20,"[enthusiasm, engagement, curiosity, passion]"
1,Why fit in..?,0,42,54,0,0,0,0,afrobeezy,0.0,...,https://video-cdn.socialverseapp.com/michael_5...,1.0,Vible,The video is structured as an emotional and in...,0.0,1.0,57,8,20,"[Anger, Determination, Pain, Empowerment, Isol..."
2,Transcending Singularity,0,20,75,0,45,8,0,afrobeezy,0.0,...,https://video-cdn.socialverseapp.com/michael_5...,1.0,Vible,The video presents a compelling visual narrati...,0.0,1.0,57,8,20,"[Wonder, Intensity, Compassion, Engagement, Co..."
3,Peak Performance?,0,71,95,6,6,28,0,afrobeezy,0.0,...,https://video-cdn.socialverseapp.com/michael_4...,1.0,Vible,The video presents a seamless narrative explor...,0.0,1.0,57,8,20,"[excitement, passion, contemplation]"
4,Our Existential Situation,0,64,72,0,9,33,0,afrobeezy,0.0,...,https://video-cdn.socialverseapp.com/michael_b...,1.0,Vible,"The video opens with a male speaker, who has a...",0.0,1.0,57,8,20,"[Concern, Passion, Reflectiveness, Urgency, Ho..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Kalsubai is the highest peak in Maharashtra, s...",0,23,54,49,2,31,0,shivam,0.0,...,https://video-cdn.socialverseapp.com/shivam_03...,1.0,Flic,I'm unable to analyze or interpret video frame...,0.0,1.0,19,8,8,"[encouraging, supportive, helpful]"
996,"Tap into the magic of winter. .Sometimes, all ...",0,7,88,53,2,23,0,shivam,0.0,...,https://video-cdn.socialverseapp.com/shivam_03...,1.0,Flic,"As the video unfolds, it presents a cohesive n...",0.0,1.0,19,8,8,"[deep contemplation, curiosity, nostalgia, awe..."
997,anzeige -- ️From idea to masterpiece ️with the,0,24,90,88,2,16,0,shivam,0.0,...,https://video-cdn.socialverseapp.com/shivam_03...,1.0,Flic,"In the video, a cohesive narrative unfolds thr...",0.0,1.0,19,8,8,"[Focused determination, Inspiring, Calmness, P..."
998,"In the heart of darkness, magic is our only sh...",0,5,83,42,2,12,0,shivam,0.0,...,https://video-cdn.socialverseapp.com/shivam_03...,1.0,Flic,The video unfolds as a cinematic narrative set...,0.0,1.0,19,8,8,"[anticipation, concern, excitement, fear, exhi..."


In [55]:
filtered_df[filtered_df['category_name'] == 'Flic']['emotions_conveyed']

filtered_df.to_csv('filtered_df.csv',index=False)

In [63]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import numpy as np
import optuna

# Assuming 'filtered_df' is the DataFrame that holds your data

# Combine title, description, and category_name for text-based features
filtered_df['combined_text'] = (
    filtered_df['title'] + ' ' +
    filtered_df['description'] + ' ' +
    filtered_df['category_name'] + ' ' +
    filtered_df['emotions_conveyed'].astype(str)
)

# Vectorize the combined text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['combined_text'])

# Normalize numerical features
numerical_columns = ['upvote_count', 'view_count', 'comment_count', 'rating_count', 'average_rating', 
                     'following', 'exit_count', 'post_count', 'share_count', 
                     'following_count', 'follower_count', 'upvoted', 'bookmarked']
scaler = StandardScaler()
filtered_df[numerical_columns] = scaler.fit_transform(filtered_df[numerical_columns])

def build_user_profile(user, df, weights):
    df = df.reset_index(drop=True)
    user_posts = df[df['username'] == user]
    weighted_vectors = []

    for index, row in user_posts.iterrows():
        interaction_weight = (
            (row['upvote_count'] * weights[0] + row['comment_count'] * weights[1] + row['view_count'] * weights[2])
            + (row['rating_count'] * row['average_rating'] * weights[3])
            + (row['post_count'] * weights[4] + row['share_count'] * weights[5])
        )
        post_vector = tfidf_matrix[index].toarray()
        weighted_vectors.append(post_vector.flatten() * interaction_weight)

    # Aggregate the weighted post vectors to build the user profile
    user_profile = np.mean(weighted_vectors, axis=0) if weighted_vectors else np.zeros(tfidf_matrix.shape[1])
    return user_profile

def get_recommendations_for_user(user, category, mood, top_n, weights):
    if user not in filtered_df['username'].unique():
        # Handle new user case
        category_filtered = filtered_df[filtered_df['category_name'].str.contains(category, case=False, na=False)]
        mood_filtered = category_filtered[category_filtered['emotions_conveyed'].str.contains(mood, case=False, na=False)]
        
        # Check if mood-based filtering returns results
        if not mood_filtered.empty:
            recommendations_df = mood_filtered
        else:
            print(f"No recommendations found for the mood '{mood}'. Showing recommendations for all moods instead.")
            recommendations_df = category_filtered

        return recommendations_df['title'].head(top_n).tolist()
    else:
        # Handle existing user case
        user_profile = build_user_profile(user, filtered_df, weights)
        cosine_similarities = cosine_similarity(user_profile.reshape(1, -1), tfidf_matrix).flatten()
        recommended_indices = cosine_similarities.argsort()[-top_n:][::-1]

        recommendations = []
        for idx in recommended_indices:
            if filtered_df.loc[idx, 'username'] != user:
                recommendations.append(filtered_df.loc[idx, 'title'])

        return recommendations

# Inputs
username = input("Enter the username: ")
category = input("Enter the category: ")
mood = input("Enter the mood: ")

# Define yhat based on the user’s posts (ground truth titles)
yhat = filtered_df[filtered_df['username'] == username]['title'].tolist() if username in filtered_df['username'].unique() else []

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(optuna_objective, n_trials=50)

# Get the best weights
optimized_weights = [study.best_params[f"weight_{i}"] for i in range(6)]

# Get recommendations based on optimized weights
recommendations = get_recommendations_for_user(username, category, mood, len(yhat) or 5, optimized_weights)

# Calculate MAE and RMSE
mae = calculate_mae(recommendations, yhat)
rmse = calculate_rmse(recommendations, yhat)

# Output the results
print(f"Optimized Weights: {optimized_weights}")
print(f"Mean Absolute Error for recommendations: {mae}")
print(f"Root Mean Squared Error for recommendations: {rmse}")
print(f"Recommendations: {recommendations}")
print(f"Ground Truth (yhat): {yhat}")

Enter the username:  joshua
Enter the category:  InstaRama
Enter the mood:  happy


[I 2024-12-11 15:06:11,920] A new study created in memory with name: no-name-8ea37515-613b-473d-9e39-a87eb14db710
[I 2024-12-11 15:06:11,927] Trial 0 finished with value: 0.0 and parameters: {'weight_0': 6.402097316005345, 'weight_1': 4.432643508087148, 'weight_2': 1.4034090742471217, 'weight_3': 9.803578106396282, 'weight_4': 8.826970043517754, 'weight_5': 2.649203259918329}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:11,933] Trial 1 finished with value: 0.0 and parameters: {'weight_0': 5.579954326507027, 'weight_1': 0.3124042346719369, 'weight_2': 4.671699797424598, 'weight_3': 5.3648431542412585, 'weight_4': 6.082892031246054, 'weight_5': 3.6077601865810136}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:11,939] Trial 2 finished with value: 0.0 and parameters: {'weight_0': 5.437061948307377, 'weight_1': 6.053367101763747, 'weight_2': 0.18558468337644185, 'weight_3': 5.392168063287736, 'weight_4': 9.363360130865036, 'weight_5': 0.8096883578003777}. Best is trial 0 wi

No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recom

[I 2024-12-11 15:06:12,151] Trial 14 finished with value: 0.0 and parameters: {'weight_0': 7.13843053155586, 'weight_1': 0.20376390723437776, 'weight_2': 5.784380513777448, 'weight_3': 1.9522346822145877, 'weight_4': 7.820514017913706, 'weight_5': 2.9390275220356066}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:12,184] Trial 15 finished with value: 0.0 and parameters: {'weight_0': 3.663879676053533, 'weight_1': 3.4739110649327096, 'weight_2': 7.452232353824898, 'weight_3': 8.309988176560253, 'weight_4': 9.713932232993468, 'weight_5': 4.572395736889636}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:12,217] Trial 16 finished with value: 0.0 and parameters: {'weight_0': 6.281077360049103, 'weight_1': 4.587248288946474, 'weight_2': 1.378351127750188, 'weight_3': 5.618869406374826, 'weight_4': 4.876912443713404, 'weight_5': 7.1520261018343545}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:12,250] Trial 17 finished with value: 0.0 and parameters: {'weight_0': 8.25454

No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.


...

No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.


[I 2024-12-11 15:06:13,050] Trial 39 finished with value: 0.0 and parameters: {'weight_0': 5.560403681081616, 'weight_1': 4.278993493002919, 'weight_2': 6.767856958935216, 'weight_3': 9.232756903730156, 'weight_4': 1.2234283315679342, 'weight_5': 3.53963598698415}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:13,089] Trial 40 finished with value: 0.0 and parameters: {'weight_0': 8.523485383633188, 'weight_1': 9.273033400090705, 'weight_2': 0.8478345186886621, 'weight_3': 0.29354172499995634, 'weight_4': 6.194866350970595, 'weight_5': 4.272640673024674}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:13,127] Trial 41 finished with value: 0.0 and parameters: {'weight_0': 4.7765779049369, 'weight_1': 2.2418170543814977, 'weight_2': 9.17876993704264, 'weight_3': 3.622943597114821, 'weight_4': 2.674220990759449, 'weight_5': 9.142220094528707}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:13,162] Trial 42 finished with value: 0.0 and parameters: {'weight_0': 0.684730115

No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.


[I 2024-12-11 15:06:13,273] Trial 45 finished with value: 0.0 and parameters: {'weight_0': 6.949723132161015, 'weight_1': 9.9607702487679, 'weight_2': 9.114053748635651, 'weight_3': 5.59962947027528, 'weight_4': 4.139881068781294, 'weight_5': 1.6770915290490596}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:13,310] Trial 46 finished with value: 0.0 and parameters: {'weight_0': 1.7447642929791685, 'weight_1': 8.181782927483763, 'weight_2': 7.3619795981615175, 'weight_3': 1.3318705692616533, 'weight_4': 7.763664496849114, 'weight_5': 3.29612170227555}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:13,346] Trial 47 finished with value: 0.0 and parameters: {'weight_0': 7.451900927048256, 'weight_1': 3.181116315853365, 'weight_2': 3.462178863145103, 'weight_3': 3.186301861577509, 'weight_4': 9.965007482806776, 'weight_5': 1.2245769039823715}. Best is trial 0 with value: 0.0.
[I 2024-12-11 15:06:13,382] Trial 48 finished with value: 0.0 and parameters: {'weight_0': 4.053595836

No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
No recommendations found for the mood 'happy'. Showing recommendations for all moods instead.
Optimized Weights: [6.402097316005345, 4.432643508087148, 1.4034090742471217, 9.803578106396282, 8.826970043517754, 2.649203259918329]
Mean Absolute Error for recommendations: 0
Root Mean Squared Error for recommendations: 0
Recommendations: ['Jai Shree Ram', 'Jai Shri Ram   Raghunandana from HanuMan 2024 movie Credits to Artists for the Artworks (Source- Pintrest)', 'flarefyfacts - 7327961407444552965', 'Jai Shree Raam', 'Ram Siya 

In [169]:
filtered_df['category_name'].value_counts()

category_name
Vible              528
E/ACC              207
InstaRama          169
SolTok              48
Pumptok             19
Flic                19
OvaDrive             5
Gratitube            2
The Igloo            1
Startup College      1
Name: count, dtype: int64

In [166]:
titles = ['Recipe for a flow state', 'Why fit in..?', 'Transcending Singularity', 'Peak Performance?', 'Our Existential Situation']
for title in titles:
    print(filtered_df[filtered_df['title']==title]['category_name'])

0    Vible
Name: category_name, dtype: object
1    Vible
Name: category_name, dtype: object
2    Vible
Name: category_name, dtype: object
3    Vible
Name: category_name, dtype: object
4    Vible
Name: category_name, dtype: object


In [ ]:
import pickle

# Define what you want to save
data_to_save = {
    "optimized_weights": optimized_weights,
    "tfidf_vectorizer": tfidf_vectorizer,
    "filtered_df": filtered_df
}

# Save the data to a .pkl file
with open('recommendation_system_data.pkl', 'wb') as file:
    pickle.dump(data_to_save, file)

print("Data saved successfully to 'recommendation_system_data.pkl'")

In [78]:
import sklearn
dir(sklearn.metrics)

['ConfusionMatrixDisplay',
 'DetCurveDisplay',
 'DistanceMetric',
 'PrecisionRecallDisplay',
 'PredictionErrorDisplay',
 'RocCurveDisplay',
 'SCORERS',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_base',
 '_classification',
 '_dist_metrics',
 '_pairwise_distances_reduction',
 '_pairwise_fast',
 '_plot',
 '_ranking',
 '_regression',
 '_scorer',
 'accuracy_score',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'auc',
 'average_precision_score',
 'balanced_accuracy_score',
 'brier_score_loss',
 'calinski_harabasz_score',
 'check_scoring',
 'class_likelihood_ratios',
 'classification_report',
 'cluster',
 'cohen_kappa_score',
 'completeness_score',
 'confusion_matrix',
 'consensus_score',
 'coverage_error',
 'd2_absolute_error_score',
 'd2_pinball_score',
 'd2_tweedie_score',
 'davies_bouldin_score',
 'dcg_score',
 'det_curve',
 'euclidean_distances',
 'explained_variance_score',
 'f1_

In [103]:
username  = 'shivam'

In [106]:
yhat = combined_df[combined_df['username'] == username]['title'].tolist()
yhat

['do it now',
 'shooting tips',
 'for immediate editing!!!!',
 'Ai viral hack!!!',
 '💥💥💥',
 '🚗 ⏩⏩',
 '🏔️🌄',
 '1 edit will be free!!!!!',
 'something different!!!!',
 '💯💯💯',
 'Peak Beauty of Nature.....................#sunsetphotography',
 'snow!!!',
 'Dancing Cat - @visiontoinspire #dodge #hellcat #charger #musclecars #4k #dodgechallanger #dodgehel...',
 'The perfect conditions in Norway .Would you take a boatride hereThe view was stunning in every direction.#drone #dji #norway',
 'Kalsubai is the highest peak in Maharashtra, standing at an impressive 1,646 meters (5,400 feet) above sea level. Located in the Western Ghats, near the town of Igatpuri, it’s known for its scenic views, challenging trekking paths, and the famous Kalsubai',
 'Tap into the magic of winter. .Sometimes, all it takes is one switch to open up a whole new world. From green hills to snow-capped peaks, and the Northern Lights dancing in the sky. Winter magic unlocked! #snowman #music.#snow #winterseason #editing',
 

In [82]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Assuming combined_df is your DataFrame with ground truth data
yhat = combined_df[combined_df['username'] == username]['title'].tolist()  # Ground truth recommendations

def calculate_mae(predictions, ground_truth):
    """
    Calculates Mean Absolute Error between the predicted and ground truth recommendation lists.
    """
    # Convert both lists to sets
    prediction_set = set(predictions)
    ground_truth_set = set(ground_truth)
    
    # Absolute error: difference in lengths after intersection
    errors = len(ground_truth_set) - len(prediction_set.intersection(ground_truth_set))
    
    # Normalize by the total number of ground truth items to get MAE
    mae = abs(errors) / len(ground_truth_set) if ground_truth_set else 0
    return mae

def calculate_rmse(predictions, ground_truth):
    """
    Calculates Root Mean Squared Error between the predicted and ground truth recommendation lists.
    """
    # Convert both lists to sets
    prediction_set = set(predictions)
    ground_truth_set = set(ground_truth)
    
    # Squared error: difference in lengths after intersection
    errors = len(ground_truth_set) - len(prediction_set.intersection(ground_truth_set))
    
    # RMSE calculation
    rmse = np.sqrt(errors ** 2 / len(ground_truth_set)) if ground_truth_set else 0
    return rmse

# Get recommendations for the user
recommendations = get_recommendations_for_user(username, top_n=len(yhat))

# Calculate Mean Absolute Error
mae = calculate_mae(recommendations, yhat)
rmse = calculate_rmse(recommendations, yhat)

# Print results
print(f"Mean Absolute Error for recommendations: {mae}")
print(f"Root Mean Squared Error for recommendations: {rmse}")

# Optionally: Display recommendations and ground truth for comparison
print(f"Recommendations: {recommendations}")
print(f"Ground Truth (yhat): {yhat}")


Mean Absolute Error for recommendations: 1.0
Root Mean Squared Error for recommendations: 1.0
Recommendations: ['Jai Shree Ram']
Ground Truth (yhat): ['Solana is heading up 📈']


In [41]:
!pip install sentence-transformers

In [42]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 0 to 999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            999 non-null    object 
 1   comment_count    999 non-null    float64
 2   upvote_count     999 non-null    float64
 3   view_count       999 non-null    float64
 4   exit_count       999 non-null    float64
 5   rating_count     999 non-null    float64
 6   average_rating   999 non-null    float64
 7   share_count      999 non-null    float64
 8   username         999 non-null    object 
 9   upvoted          999 non-null    float64
 10  bookmarked       999 non-null    float64
 11  thumbnail_url    999 non-null    object 
 12  following        999 non-null    float64
 13  category_name    999 non-null    object 
 14  description      999 non-null    object 
 15  isVerified       999 non-null    float64
 16  has_wallet       999 non-null    float64
 17  post_count       999 

In [43]:
filtered_df

,title,comment_count,upvote_count,view_count,exit_count,rating_count,average_rating,share_count,username,upvoted,...,thumbnail_url,following,category_name,description,isVerified,has_wallet,post_count,following_count,follower_count,combined_text
0,Recipe for a flow state,-0.209867,0.728560,0.083248,-0.365784,0.701225,1.801194,7.875664,afrobeezy,3.854496,...,https://video-cdn.socialverseapp.com/michael_9...,2.965488,Vible,The video unfolds as a dynamic exploration of ...,0.0,1.0,-2.826908,-2.492995,-2.314324,Recipe for a flow state The video unfolds as a...
1,Why fit in..?,-0.209867,0.584881,-0.158744,-0.365784,-0.439428,-0.726536,-0.178374,afrobeezy,-0.259437,...,https://video-cdn.socialverseapp.com/michael_5...,2.965488,Vible,The video is structured as an emotional and in...,0.0,1.0,-2.826908,-2.492995,-2.314324,Why fit in..? The video is structured as an em...
2,Transcending Singularity,-0.209867,-0.468768,0.405904,-0.365784,6.893341,-0.164818,-0.178374,afrobeezy,-0.259437,...,https://video-cdn.socialverseapp.com/michael_5...,2.965488,Vible,The video presents a compelling visual narrati...,0.0,1.0,-2.826908,-2.492995,-2.314324,Transcending Singularity The video presents a ...
3,Peak Performance?,-0.209867,1.973782,0.943665,-0.209389,0.538275,1.239476,-0.178374,afrobeezy,-0.259437,...,https://video-cdn.socialverseapp.com/michael_4...,2.965488,Vible,The video presents a seamless narrative explor...,0.0,1.0,-2.826908,-2.492995,-2.314324,Peak Performance? The video presents a seamles...
4,Our Existential Situation,-0.209867,1.638530,0.325240,-0.365784,1.027126,1.590550,-0.178374,afrobeezy,-0.259437,...,https://video-cdn.socialverseapp.com/michael_b...,2.965488,Vible,"The video opens with a male speaker, who has a...",0.0,1.0,-2.826908,-2.492995,-2.314324,Our Existential Situation The video opens with...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Kalsubai is the highest peak in Maharashtra, s...",-0.209867,-0.325088,-0.158744,0.911447,-0.113527,1.450120,-0.178374,shivam,-0.259437,...,https://video-cdn.socialverseapp.com/shivam_03...,2.965488,Flic,I'm unable to analyze or interpret video frame...,0.0,1.0,-2.962312,-2.492995,-3.492918,"Kalsubai is the highest peak in Maharashtra, s..."
996,"Tap into the magic of winter. .Sometimes, all ...",-0.209867,-1.091378,0.755449,1.015710,-0.113527,0.888403,-0.178374,shivam,-0.259437,...,https://video-cdn.socialverseapp.com/shivam_03...,2.965488,Flic,"As the video unfolds, it presents a cohesive n...",0.0,1.0,-2.962312,-2.492995,-3.492918,"Tap into the magic of winter. .Sometimes, all ..."
997,anzeige -- ️From idea to masterpiece ️with the,-0.209867,-0.277195,0.809225,1.928018,-0.113527,0.396899,-0.178374,shivam,-0.259437,...,https://video-cdn.socialverseapp.com/shivam_03...,2.965488,Flic,"In the video, a cohesive narrative unfolds thr...",0.0,1.0,-2.962312,-2.492995,-3.492918,anzeige -- ️From idea to masterpiece ️with the...
998,"In the heart of darkness, magic is our only sh...",-0.209867,-1.187165,0.621009,0.728985,-0.113527,0.116041,-0.178374,shivam,-0.259437,...,https://video-cdn.socialverseapp.com/shivam_03...,2.965488,Flic,The video unfolds as a cinematic narrative set...,0.0,1.0,-2.962312,-2.492995,-3.492918,"In the heart of darkness, magic is our only sh..."


In [44]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import numpy as np


filtered_df['combined_text'] = filtered_df['title'] + ' ' + filtered_df['description'] + ' ' + filtered_df['category_name']

# TF-IDF Vectorizer to convert text into vectors
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['combined_text'])

# Step 2: Standardize Numerical Columns
numerical_columns = ['upvote_count', 'view_count', 'comment_count', 'following']
scaler = StandardScaler()
filtered_df[numerical_columns] = scaler.fit_transform(filtered_df[numerical_columns])

def build_user_profile(user, df):
    # Filter posts that the user has interacted with (liked, commented, etc.)
    user_posts = df[df['username'] == user]
    
    # Aggregate TF-IDF vectors of liked/commented/viewed posts
    weighted_vectors = []
    
    for _, row in user_posts.iterrows():
        # Combine different interaction metrics like upvotes, comments, views, ratings, new ones like exit_count, share_count, etc.
        interaction_weight = (
            (row['upvote_count'] + row['comment_count'] + row['view_count']) / 3  # Upvotes, comments, and views are averaged
            + (row['rating_count'] * row['average_rating'])  # Rating count * average rating to factor in rating interaction
            + (row['exit_count'] + row['post_count'] + row['share_count']) / 3  # Include exit, post, and share counts
            + (row['upvoted'] + row['bookmarked']) / 2  # Add upvoted and bookmarked status
            + (row['following_count'] + row['follower_count']) / 2  # Factor in following and follower counts
        ) / 6  # Normalize the weight to keep it balanced
        
        # Find the position of this post in the original DataFrame
        post_index = df[df['title'] == row['title']].index[0]  # Find the index of the current post
        
        post_vector = tfidf_matrix[post_index].toarray()  # Convert the sparse matrix to dense
        weighted_vectors.append(post_vector.flatten() * interaction_weight)  # Flatten and weight the vector
        
    if weighted_vectors:
        user_profile = np.mean(weighted_vectors, axis=0)  # Average the weighted vectors
    else:
        user_profile = np.zeros(tfidf_matrix.shape[1])  # If no posts interacted with, return a zero vector
    
    return user_profile


# Step 4: Compute Cosine Similarity and Generate Recommendations
def get_recommendations_for_user(user, top_n=5):
    user_profile = build_user_profile(user, filtered_df)
    
    # Compute cosine similarities between the user's profile and all posts
    cosine_similarities = cosine_similarity(user_profile, tfidf_matrix).flatten()
    
    # Get indices of the top_n most similar posts (excluding posts the user has already interacted with)
    recommended_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # Filter out posts the user has already interacted with
    recommendations = []
    for idx in recommended_indices:
        if filtered_df.loc[idx, 'username'] != user:  # Ensure the post is not from the same user
            recommendations.append(filtered_df.loc[idx, 'title'])
    
    return recommendations

# Example usage: Get recommendations for a specific user (replace 'user123' with an actual username)
username = 'shivam'
top_5_recommendations = get_recommendations_for_user(username, top_n=5)
print(f"Top 5 recommendations for {username}: {top_5_recommendations}")


IndexError: row index (999) out of range

In [ ]:
from transformers import BertTokenizer,BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch 

def text_to_bert(text_data):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    
    embeddings = []
    for text in text_data:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state[0, 0, :].numpy())
    
    return np.array(embeddings)

    

In [ ]:
filtered_df

In [ ]:
combined_features.shape

In [ ]:
cosine_sim = cosine_similarity(combined_features)

In [ ]:
def recommend_posts(title, cosine_sim, dataset, combined_features):
    indices = pd.Series(dataset.index, index=dataset['title']).drop_duplicates()
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    post_indices = [i[0] for i in sim_scores]
    return dataset.iloc[post_indices]

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def recommend_posts_with_predictions(title, cosine_sim, dataset, combined_features):
    indices = pd.Series(dataset.index, index=dataset['title']).drop_duplicates()
    idx = indices[title]
    
    # Get similarity scores between the post and all others
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort by similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top 5 most similar posts (skip the first one, as it is the same post)
    sim_scores = sim_scores[1:6]
    
    post_indices = [i[0] for i in sim_scores]
    recommended_posts = dataset.iloc[post_indices]
    
    # Predict ratings based on cosine similarity scores
    predicted_ratings = [dataset.iloc[i]['average_rating'] * sim_score[1] for i, sim_score in zip(post_indices, sim_scores)]
    
    # Return recommended posts along with their predicted ratings
    recommended_posts['predicted_rating'] = predicted_ratings
    return recommended_posts

# Function to calculate MAE and RMSE for a batch of recommendations
def evaluate_recommendations(title, cosine_sim, dataset, combined_features):
    recommended_posts = recommend_posts_with_predictions(title, cosine_sim, dataset, combined_features)
    
    # Actual ratings from the dataset
    actual_ratings = recommended_posts['average_rating']
    
    # Predicted ratings (calculated based on similarity scores)
    predicted_ratings = recommended_posts['predicted_rating']
    
    # Calculate MAE
    mae = mean_absolute_error(actual_ratings, predicted_ratings)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
    
    return mae, rmse

In [ ]:
# Test title
sample_title = "Our Existential Situation"

# Evaluate recommendations using MAE and RMSE
mae, rmse = evaluate_recommendations(sample_title, cosine_sim, filtered_df, combined_features)

# Output the results
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

In [ ]:
from IPython.display import Markdown, display

# Test title
sample_title = "Our Existential Situation"

# Generate Recommendations
recommendations = recommend_posts(sample_title, cosine_sim, filtered_df)

# Display Recommendations
display(Markdown(f"### Recommendations for: {sample_title}"))
for index, row in recommendations.iterrows():
    display(Markdown(f"**Title:** {row['title']}"))
    display(Markdown(f"**Description:** {row['description']}"))
    display(Markdown(f"**Category:** {row['category_name']}"))
    display(Markdown(f"**Average Rating:** {row['average_rating']}"))
    display(Markdown("---"))

In [ ]:
filtered_df.info()

In [ ]:
filtered_df = filtered_df.drop(columns = ['title','category_name','description'],axis = 1)
filtered_df

In [ ]:
from sklearn.preprocessing import StandardScaler 

# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(filtered_df)
# scaled_data
filtered_df.info()

In [ ]:
Liked['liked_at'] = pd.to_datetime(Liked['liked_at'])
Rated['rated_at'] = pd.to_datetime(Rated['rated_at'])
viewed['viewed_at'] = pd.to_datetime(viewed['viewed_at'])

#  The disparity arises because a single user can interact with multiple posts in various ways (e.g., liking, rating, or viewing). This results in a one-to-many relationship between user_id and post_id in datasets like Liked, Rated, and Viewed. This behavior is expected in datasets capturing user interactions.